<a href="https://colab.research.google.com/github/edgeofchaos42/ComplexityExplorer/blob/main/Session_11_Traders_move_part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1- Review lesson 10 and intro

Script Notes: 

In lesson 10 we added the welfare function using the cobb-douglas functional from to assess which cell would give the agent the most benefit. In this lesson we will now move to a specific cell that provides the most benefit.    

Remember if you shut down your colab kernel you will need to add the sugar-map.txt file again or you can mount your drive and not have to load it every time   

*Review each class and model set up*

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
try: 
  import mesa
except: 
  !pip install mesa --quiet
  import mesa
import numpy as np
import math
import matplotlib.pyplot as plt



%matplotlib inline 

     |████████████████████████████████| 2.5 MB 7.0 MB/s 
     |████████████████████████████████| 596 kB 48.7 MB/s 
     |████████████████████████████████| 64 kB 2.4 MB/s 


In [11]:
def get_distance(pos_1, pos_2):
    """Calculate Euclidean distance between two positions.
    Args:
        pos_1, pos_2: Coordinate tuples for both points.
    
    used in trader.move()
    """
    x1, y1 = pos_1
    x2, y2 = pos_2
    dx = x1 - x2
    dy = y1 - y2
    return math.sqrt(dx**2 + dy**2)

In [3]:
class Sugar(mesa.Agent):
    """
    Sugar is a FSM that
    - contains an amount of sugar
    - grows 1 amount of sugar at each turn (rule G1).
    """
    
    
    def __init__(self, unique_id, model, pos, max_sugar): 
      super().__init__(unique_id, model) #part 3
      self.pos = pos
      self.amount = max_sugar
      self.max_sugar = max_sugar

    def step(self): #Part 1
      self.amount = min([self.max_sugar,self.amount+1])
      
      

In [6]:
class Spice(mesa.Agent):
    """
    Spice is a FSM that
    - contains an amount of spice
    - grows 1 amount of spice at each turn. (rule G1)
    """

    def __init__(self, unique_id, model, pos, max_spice): 
        super().__init__(unique_id, model) 
        self.pos = pos
        self.amount = max_spice
        self.max_spice = max_spice

    def step(self): #Part 1
      self.amount = min([self.max_spice,self.amount+1])
       

## Add cell move 

**Part 1**

------------

We now have list of every cell in our vision that does not already have an agent on it and we know how much welfare it will provide for us. What we want to do is maximize the welfare.

so first we need to identify the max amout of welfare in our list, we can do this by just using the max function in python

now because there is not much variance in the landscape each sugar and spice agent ranges from 0 to 4 there may be multiple cells that provide similar welfare and we want to move to the closest. In addition, we need to know the cell location to move to and not the max welfare so we will do two list comprehensions to get the cell location of all the cells providng welfare to us.  To do this we will use pythons [math libray and the close function](https://docs.python.org/3/library/math.html). so first we need to import math and then use the kewy word arguemnt or kward to make the function to 2 decimal places


 We are doing this because we are using floating point numbers and so minor variations at the 9 decimal place are too specific for this model. This detail is important as more than one modeler has seen seemingly critical results disappear when they realized they were comparing to floating point numbers.  

It is important to emphasize we are using the dynamics of python data structures to get welfare and get the cell location. Welfare and neighbors are the same length just representing the cell in two different views - neighbors is the position, welfare is the welfare calcualtion from the agents unique situation, as they are ordered lists  the 0 position in the list is the same as the zero index in the welfare list so we can just use the index to extract the positions. 

Lets see this with a print statement and as you can see there isnt typically a cell but mulitple cells that provide similar welfare. Now that we have a list of indices we can get the position of those optimal moves.


**Part 2**

------------

Now that we have the positions we can move per Growing artifical socities to the nearest cell. Although not in this model, we could even create a function that the metabolism or the amount of energy burned is variable based on the distance moved. 

However in this case we will just move to the closest cell and every move will consume the same amount of energy. 

Again there are several ways to do this, but in this case we are going to calucate the euclidean distance using the pyhtagorem theorem and as this is a universal calculation and not specific to a trader we will put it in its own cell. Again using list comprehension we will create the distance to each cell 
and figure out the minimum then using the isclose function again arbitrarily set to 2 decimal places we will then keep only those locations that are close

then we test to make sure it is doing what we think it is doing. 

*comment on results*

then we are going to shuffle the list to maintain the randomness in the model and then 

then we move to the first one. To do this we will use mesa's [move_agent function](https://github.com/projectmesa/mesa/blob/1ad2456cb812a78fbddd996cc246485fdca58607/mesa/space.py#L400) so we reference the model the grid and then move_agent. 


This then concludes session 11 as well as the the move function. Fortunately some of the functions we built here can be used in our functions of eating and trading. 

In the next session we eat. 






In [15]:
class Trader(mesa.Agent):
    """
    TraderAgent is a 
    - has a metabolism for sugar and spice
    - harvest and trades sugar and spice to survive and thrive
    """

    def __init__(self, unique_id, model,pos,moore=False, sugar=0, 
                 spice=0, metabolism_sugar=0, metabolism_spice=0,
                 vision=0): 
      super().__init__(unique_id, model)
      self.pos = pos 
      self.moore = False 
      self.sugar = sugar 
      self.spice = spice 
      self.metabolism_sugar=metabolism_sugar 
      self.metabolism_spice=metabolism_spice 
      self.vision = vision 

  
    def get_sugar(self, pos): #part 2
      '''
      used in get_sugar_amount
      '''
      this_cell = self.model.grid.get_cell_list_contents(pos)
      for agent in this_cell:
          if type(agent) is Sugar:
              return agent
      return None
    
    
    def get_sugar_amount(self, pos):
      '''
      used in move()
      '''
      
      sugar_patch = self.get_sugar(pos)
      if sugar_patch:
          return sugar_patch.amount
      return 0
    
    
    def get_spice(self, pos):
      '''
      used in get_spice_amount
      '''
      this_cell = self.model.grid.get_cell_list_contents(pos)
      for agent in this_cell:
          if type(agent) is Spice:
              return agent
      return None
        
    
    def get_spice_amount(self, pos):
        '''
        used in move()
        '''

        spice_patch = self.get_spice(pos)
        if spice_patch: 
          return spice_patch.amount
        return 0

    
    def is_occupied(self, pos): 
      '''
      Helper function for move
      '''
      
      this_cell = self.model.grid.get_cell_list_contents(pos)
      for a in this_cell:
          if isinstance(a, Trader) and a.pos != self.pos:
              return True
      return False
       
    def calculate_welfare(self, sugar, spice): #This will be built to be like the exmaple later with none, but right now it wouldn't make any sense
      '''
      Helper function for move
      '''

      #calculate total resources
      m_total = self.metabolism_sugar + self.metabolism_spice
      #Cobb-Douglas functional form
      return sugar** (self.metabolism_sugar/m_total) * spice ** (
          self.metabolism_spice/m_total)
    
    
    def move(self): 
           
      # 1. Get neighbors within vision

      neighbors = [i 
                   for i in self.model.grid.get_neighborhood(
                       self.pos, self.moore,True, self.vision
                       )
                   if not self.is_occupied(i)
      ]

      # 2. Find the patch which porduce the maximum welfare. 

      welfares = [
            self.calculate_welfare(
                self.sugar + self.get_sugar_amount(pos),
                self.spice + self.get_spice_amount(pos),
            )
            for pos in neighbors] #part 2

      #find the highest welfare in the cell 
      max_welfare = max(welfares)
      #get the index of maximal welfare cells
      candidate_indices = [i for i in range(len(welfares))
                             if math.isclose(welfares[i], max_welfare,
                                             rel_tol=1e-02)]
      #convert index to positions of those cells
      candidates = [neighbors[i] for i in candidate_indices]
     
      
      # 3. Find the nearest patch among the candidate.
      min_dist = min([get_distance(self.pos, pos) for pos in candidates])
      final_candidates = [
          pos
          for pos in candidates
          if math.isclose(get_distance(self.pos, pos),min_dist, rel_tol=1e-02)
      ]
      self.random.shuffle(final_candidates)

      # 4. Move agent.
      self.model.grid.move_agent(self, final_candidates[0])


      #print(max_welfare, candidates) #part 1
      print(min_dist, final_candidates) #part 2



In [13]:
class SugarscapeG1mt(mesa.Model):

  def __init__(self, width=50, height=50, initial_population=100,
               endowment_min =25, endowment_max =50, metabolism_min = 1,
                 metabolism_max = 5, vision_min = 1, vision_max =5): 
    
    self.width = width
    self.height = height
    self.initial_population =initial_population 
    self.endowment_min = endowment_min 
    self.endowment_max = endowment_max 
    self.metabolism_min = metabolism_min
    self.metabolism_max = metabolism_max
    self.vision_min = vision_min
    self.vision_max = vision_max


    self.schedule = mesa.time.RandomActivationByType(self)
    self.grid = mesa.space.MultiGrid(self.width, self.height, torus=True)

    sugar_distribution = np.genfromtxt("/content/drive/MyDrive/sugar-map.txt") 
    spice_distribution = spice = np.flip(sugar_distribution,1)
    
    #ensure unique id
    agent_id = 0
    for _, x, y in self.grid.coord_iter(): 
      max_sugar = sugar_distribution[x,y]
      if max_sugar > 0: 
        sugar = Sugar(agent_id, self, (x,y),max_sugar)
        self.grid.place_agent(sugar, (x, y))
        #Place Agent
        self.schedule.add(sugar) 
        agent_id += 1

      max_spice = spice_distribution[x,y]
      if max_spice > 0: 
        spice = Spice(agent_id, self, (x,y), max_spice)
        self.grid.place_agent(spice,(x,y))
        self.schedule.add(spice)
        agent_id += 1
         
    
    for i in range(self.initial_population):
      #get agent position
      x = self.random.randrange(self.width) 
      y = self.random.randrange(self.height) 
      # See GAS page 108 for parameters initialization.
      # Each agent is endowed by a random amount of sugar and spice
      sugar = self.random.uniform(self.endowment_min, self.endowment_max+1) 
      spice = self.random.uniform(self.endowment_min, self.endowment_max+1) 
      #add metabolism also on page #108
      metabolism_sugar = self.random.uniform(self.metabolism_min, self.metabolism_max)
      metabolism_spice = self.random.uniform(self.metabolism_min, self.metabolism_max)
      #add vision
      vision =int(self.random.uniform(self.vision_min, self.vision_max))
      trader = Trader(
                agent_id,
                self,
                (x, y),
                False,
                sugar,
                spice,
                metabolism_sugar,
                metabolism_spice,
                vision,
            )
      self.grid.place_agent(trader, (x, y))
      self.schedule.add(trader)
      agent_id += 1

  def step(self): 
    
    for sugar in self.schedule.agents_by_type[Sugar].values(): 
      sugar.step()
    
    for spice in self.schedule.agents_by_type[Spice].values(): 
      spice.step()
    
    Traders = self.schedule.agents_by_type[Trader].values()
    Trader_shuffle = list(Traders)
    self.random.shuffle(Trader_shuffle)
    for agent in Trader_shuffle: 
      agent.move()  
    
    self.schedule.time +=1
    self.schedule.steps += 1
    

  
  def run_model(self, step_count=1000):
    
    for i in range(step_count):
      self.step()
      

    #self.schedule.step()_

        

## Model Run 


In [16]:
model = SugarscapeG1mt()
model.run_model(step_count=1)



0.0 [(7, 12)]
1.4142135623730951 [(35, 18)]
0.0 [(41, 8)]
0.0 [(13, 13)]
0.0 [(30, 17)]
0.0 [(31, 18)]
1.0 [(32, 16)]
2.23606797749979 [(42, 21)]
1.0 [(20, 8)]
0.0 [(9, 21)]
3.1622776601683795 [(41, 43)]
0.0 [(13, 35)]
3.0 [(25, 22)]
0.0 [(11, 36)]
0.0 [(2, 15)]
0.0 [(6, 19)]
2.0 [(32, 25)]
0.0 [(28, 41)]
0.0 [(47, 8)]
1.0 [(43, 27), (42, 28)]
2.23606797749979 [(16, 21)]
2.23606797749979 [(29, 25)]
2.23606797749979 [(12, 16)]
0.0 [(46, 18)]
0.0 [(19, 27)]
2.8284271247461903 [(23, 3)]
0.0 [(26, 11)]
2.0 [(44, 39)]
0.0 [(39, 14)]
0.0 [(47, 18)]
0.0 [(16, 3)]
1.0 [(43, 41)]
0.0 [(46, 42)]
0.0 [(8, 42)]
1.0 [(45, 32), (44, 31)]
3.0 [(29, 33)]
0.0 [(33, 40)]
0.0 [(43, 16)]
0.0 [(13, 47)]
0.0 [(26, 13)]
2.0 [(32, 39)]
0.0 [(2, 3)]
1.4142135623730951 [(2, 45)]
1.0 [(11, 38), (12, 39)]
1.0 [(42, 22), (41, 23)]
1.0 [(24, 33)]
0.0 [(10, 32)]
46.0 [(2, 6)]
1.4142135623730951 [(14, 34)]
0.0 [(31, 5)]
0.0 [(33, 1)]
0.0 [(36, 44)]
0.0 [(17, 25)]
0.0 [(30, 11)]
0.0 [(44, 17)]
0.0 [(24, 20)]
1.4142135